# Movie Recommender Notebook 

## 介绍

欢迎！在此笔记本中，您将分析数据集并在此创建电影推荐器应用程序。这里的代码和数据集已为您编写并预加载。您所要做的就是选择并运行每个单元格--通过菜单栏下方的“运行”按钮或使用** Shift + Enter **。

您的数据集应包含：

* 用户评分数据-rating.csv
* 具有20种独特流派的电影标题-movie.csv
* 链接到imdb-links.csv

您将使用Amazon SageMaker的Factorization Machines算法，特别是二进制分类器方法来训练您的模型。训练完模型后，您将使用Amazon SageMaker终端节点进行部署，并使用模型的终端节点构建一个简单的电影推荐应用程序。

分解机（Factorization Machines）是一种通用的有监督学习算法，您可以将其用于分类和回归任务。它是线性模型的扩展，旨在简化捕捉高维稀疏数据集中要素之间的交互。例如，在点击预测系统中，因式分解机器模型可以捕获将来自某个广告类别的广告放置在来自某个页面类别的页面上时观察到的点击率模式。因数分解机是处理高维稀疏数据集的任务（例如，点击预测和项目推荐）的理想选择。

Amazon SageMaker的分解机算法提供了该算法的强大，高度可扩展的实现，在广告点击预测和推荐器系统中已变得非常流行。

首先，我们需要通过一些先决条件步骤（包括权限，配置等）来设置环境。


In [ ]:
bucket = 'S3bucket'
prefix = 'sagemaker/movielens'
 
# 定义IAM role
import boto3
import re
import seaborn as sns
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
%matplotlib inline

role = get_execution_role()

In [ ]:
# 定义本地目录和本地文件以进行数据挖掘
dirName = 'firstname'
files = ['movies.csv', 'ratings.csv', 'links.csv']
movies = pd.read_csv(dirName+'/'+files[0])
ratings = pd.read_csv(dirName+'/'+files[1])
links = pd.read_csv(dirName+'/'+files[2])

## 数据准备与数据探索

In [ ]:
movies.sample(5)

In [ ]:
ratings.sample(5)

In [ ]:
links.sample(5)

### 安装 wordcloud

In [ ]:
!conda install -c conda-forge wordcloud --yes

### 电影标题 wordcloud
电影标题中是否有某些单词出现频率更高？让我们尝试对两个电影标题都使用词云可视化方法来解决这个问题。
这里我们会应用到可视化python包wordcloud，wordcloud是优秀的词云展示第三方库，以词语为基本单位，通过图形可视化的方式，更加直观和艺术的展示文本。

In [ ]:
import wordcloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# 创建电影标题的词云
movies['title'] = movies['title'].fillna("").astype('str')
title_corpus = ' '.join(movies['title'])
title_wordcloud = WordCloud(stopwords=STOPWORDS, background_color='black', height=2000, width=4000).generate(title_corpus)

# 绘制到词云
plt.figure(figsize=(16,8))
plt.imshow(title_wordcloud)
plt.axis('off')
plt.show()

### wordcloud 类型
由于类型变量描述了电影的内容（例如动画，恐怖，科幻），因此在构建推荐引擎时，流派变量肯定很重要。基本假设是，同一类型的电影应具有相似的内容。让我们尝试确切地了解哪种类型最受欢迎。

In [ ]:
# 对类型关键字进行普查
genre_labels = set()
for s in movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

# 计算每个类型关键字出现次数的函数。
def count_word(dataset, ref_col, census):
    keyword_count = dict()
    for s in census: 
        keyword_count[s] = 0
    for census_keywords in dataset[ref_col].str.split('|'):        
        if type(census_keywords) == float and pd.isnull(census_keywords): 
            continue        
        for s in [s for s in census_keywords if s in census]: 
            if pd.notnull(s): 
                keyword_count[s] += 1
    #_____________________________________
    # 将列表中的字典转换为按频率对关键字进行排序
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count

# 调用此函数可以访问按降低频率排序的类别关键字列表
keyword_occurences, dum = count_word(movies, 'genres', genre_labels)
keyword_occurences[:5]

In [ ]:
# 定义用于产生类型词云的字典
genres = dict()
trunc_occurences = keyword_occurences[0:18]
for s in trunc_occurences:
    genres[s[0]] = s[1]

# 创建 wordcloud
genre_wordcloud = WordCloud(width=1000,height=400, background_color='white')
genre_wordcloud.generate_from_frequencies(genres)

# 划分 wordcloud
f, ax = plt.subplots(figsize=(16, 8))
plt.imshow(genre_wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

### 一种热编码

In [ ]:
genres_raw = list(movies['genres'])

In [ ]:
genres_cooked = [x.split('|') for x in genres_raw]

In [ ]:
genres_cooked[:10]

In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
oneHotEncoded = mlb.fit_transform(genres_cooked)

In [ ]:
oneHotEncoded = pd.DataFrame(oneHotEncoded)
oneHotEncoded.sample(10)

In [ ]:
# 这20种一键编码的功能代表以下20种类型：
#
# 'Adventure', 'Comedy', 'Action', 'Drama', 'Crime', 'Children',
# 'Mystery', 'Documentary', 'Animation', 'Thriller', 'Horror',
# 'Fantasy', 'Film-Noir', 'Western', 'Romance', 'Sci-Fi', 'Musical',
# 'War', 'IMAX', '(no genres listed)'

In [ ]:
# 现在，我们需要一个具有以下结构的矩阵：
# movieId, movieTitle, corresponding one-hot-encoded genre
# 简而言之，我们需要从电影中删除流派列，然后添加新的数据框。
newDf = pd.concat(axis=1, objs=(movies, oneHotEncoded))

In [ ]:
newDf.drop(columns=['genres'], inplace=True)

In [ ]:
newDf.head(10)

In [ ]:
# 现在进入评分文件，
# 我们需要创建一个numOfUsersxnumOfMovies
# 矩阵以查看哪个用户对哪个电影进行了评级。
# 我们将去掉timestamp列，因为它不是必需的。

In [ ]:
ratings.drop(columns=['timestamp_c'], inplace=True)

In [ ]:
ratings.sample(5)

In [ ]:
ratings.describe()

In [ ]:
# 我们需要注意的一个问题是，原始电影数据包含所有用户未评级的电影。因此，我们需要删除未分级的电影。
ratedMovieIDs = list(ratings.movieid.unique())

In [ ]:
print("Unrated movies: {}".format(len(movies['movieid'])-len(ratedMovieIDs)))

In [ ]:
# 那就是矛盾之处。我们需要删除多余的电影.

In [ ]:
# 我们最新开发的数据框是newDf，因此我们将使用它代替电影DF
newDf = newDf[newDf['movieid'].isin(ratedMovieIDs)]

In [ ]:
len(newDf['movieid']), len(ratedMovieIDs)
# 现在，我们看到了newDf ['movieId']的长度以及额定电影的长度。两者应该相同。

In [ ]:
nbUsers = ratings['userid'].max()
nbMovies = ratings['movieid'].max()
# 由于还添加了一种热编码的类型特征，因此增加了20个，为IMDB和TMDB ID添加了2个。
nbFeatures = nbUsers + nbMovies + 20
print("Number of Users: %d" % nbUsers)
print("Number of Movies: %d" % nbMovies)
print("Number of Features: %d" % nbFeatures)

## 可视化分析

In [ ]:
movies_by_user = ratings.groupby('userid')
res = movies_by_user['movieid'].count().hist(bins=100)
res.set_title('Rating count distribution')
res

In [ ]:
# 显然，用户对其评级非常大方，这就是为什么大多数电影都获得4星评级的原因。
ratings['rating'].hist()

In [ ]:
import csv
from scipy.sparse import lil_matrix

# 为每个用户建立一个分级电影列表。
# 我们需要添加随机负样本。
moviesByUser = {}
for userId in range(nbUsers):
    moviesByUser[str(userId)] = []

for (userId, movieId, rating) in ratings.values:
    moviesByUser[str(int(userId) - 1)].append(int(movieId) - 1)

### 二元推荐引擎

我们将构建一个二元制推荐器（即，喜欢/不喜欢）。将2.5星或更高的评级设置为1。将更低的评级设置为0。除此之外，我们还将在每个输入向量的末尾附加一个热编码的向量。

In [ ]:
def loadDataset(dataframe, lines, columns):
    # 特征是在稀疏矩阵中一键编码
    X = lil_matrix((lines, columns)).astype('float32')
    # 标签存储在向量中
    Y = []
    line = 0
    # 下面的for循环对用户和电影执行一键编码
    for (userid, movieid, rating) in dataframe.values:
        ohe = (newDf[newDf.movieid == movieid][list(range(20))].values)[0]
        X[line, int(userid) - 1] = 1
        # 如果您是从describe（）函数返回的，那么这里是有关movieID索引编制的知识。下面的代码行表示电影ID的编码从最后一个用户列之后立即开始。
        # 这就是为什么我们要添加具有movieID的用户数量的原因。
        

        # 让我举例说明。
        # 考虑到我们需要创建一个稀疏向量，其中用户ID为1的用户对电影标识符为movieId1的电影进行评级。在这种情况下，X [line]的第零索引将设置为1，X [line]的第671st索引将被设置]将设置为1。
        # X [line]的索引从0到670表示671个用户，索引671到164618的表示电影。
        
        # 另一个示例如下所示：
        # 如果具有userId 77的用户对带有movieId 200的电影进行评级，则以下2个索引将被标记为1：索引号76，索引号[671+（200-1）] =880。这是基本公式。
        X[line, int(nbUsers) + int(movieid)-1] = 1
        # 此内部循环将单热编码的类型迭代添加到输入的稀疏矩阵中。 20，因为类型数。每个流派组成一列。
        for i in range(20):
            X[line, int(columns)-20+(i)] = ohe[i]

        # 还要附加ID
#         X[line, int(columns) - 2] = imdbid
#         X[line, int(columns) - 1] = tmdbid
        # 可以测试非2.5的值以查看不同的结果
        if int(rating) >= 2.5:
            Y.append(1)
        else:
            Y.append(0)
        line = line + 1
            
    Y = np.array(Y).astype('float32')
    return X, Y

In [ ]:
%%time

nbRatingsTrain = len(ratings)
X_train, y_train = loadDataset(ratings, nbRatingsTrain, nbFeatures)

In [ ]:
X_test = X_train[-500:]
y_test = y_train[-500:]

In [ ]:
X_test.shape

In [ ]:
# 在S3存储桶上指定训练，测试和输出结果将被上传的位置
train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

output_location  = 's3://{}/{}/output'.format(bucket, prefix)

## 上传训练数据
Now that we've prepared our data, we'll need to convert it into recordIO-wrapped protobuf and upload it to S3 for Amazon SageMaker.
现在，我们已经准备好数据，我们需要将其转换为recordIO-wrapped的protobuf，并将其上传到Amazon SageMaker的S3。

In [ ]:
import sagemaker.amazon.common as smac
import io

def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, y_test, bucket, test_prefix, test_key)    

print('uploaded training data location: {}'.format(train_data))
print('uploaded test data location: {}'.format(test_data))
print('training artifacts will be uploaded to: {}'.format(output_location))

## 训练分解机模型
一旦我们对数据进行了预处理并以正确的格式进行训练后，下一步就是使用数据实际训练模型。

我们将使用Amazon SageMaker Python SDK进行培训并监控状态，直到完成为止。尽管数据集很小，但培训应该花费5到7分钟，置备硬件和加载算法容器需要花费一些时间。

首先，让我们指定我们的容器。由于我们希望此笔记本在Amazon SageMaker的所有8个地区中运行，因此我们将创建一个小查找。可以在AWS文档中找到有关算法容器的更多详细信息。

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'factorization-machines')



接下来，我们将启动基本估计量，确保传递必要的超参数。注意：

* Feature_dim设置为要素总数，即2625。

* Predictor_type设置为'binary_classifier'，因为我们试图预测用户是否会喜欢这部电影。

* Mini_batch_size设置为1000。可以调整此值以在拟合和速度上进行相对较小的改进，但是在大多数情况下，相对于数据集选择一个合理的值是合适的。

* Num_factors设置为64。如前所述，分解机器为所有功能找到了较低维度的交互表示。减小此值可提供更简化的模型，更接近线性模型，但可能会牺牲有关交互的信息。将其放大可提供特征交互的更高维度表示，但会增加计算复杂度并可能导致过拟合。在实际应用中，应花费时间将此参数调整为适当的值。

* Epochs设置为100，这是要运行的训练Epochs的数量。时期是所有训练向量一次用于更新权重的次数的度量。

In [ ]:
import sagemaker

sess = sagemaker.Session()

fm = sagemaker.estimator.Estimator(container,
                                   role, 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c5.xlarge',
                                   output_path=output_location,
                                   sagemaker_session=sess)

fm.set_hyperparameters(feature_dim=nbFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm.fit({'train': train_data, 'test': test_data})

训练和验证完成后，您实际上可以在训练输出中看到模型的准确性- **binary_classification_accuracy**. 该值应为80％或更高。

## 设置模型托管

现在我们已经训练了模型，我们可以将其部署在Amazon SageMaker实时托管终端节点之后。这将允许从模型中动态地做出预测（或推断）。

请注意，如果模型创建的目标是AWS Lambda，AWS Greengrass，Amazon Redshift，Amazon Athena或其他部署目标，则Amazon SageMaker允许您灵活地导入在其他地方训练过的模型，以及选择不导入模型的选择。

In [ ]:
fm_predictor = fm.deploy(initial_instance_count=1,
                         instance_type='ml.c5.xlarge')

## 模型应用-电影推荐器

要使用我们的模型，我们可以将HTTP POST请求传递到端点以获取预测。但是，为了简化操作，我们将再次使用Amazon SageMaker Python SDK并指定如何序列化请求和反序列化特定于该算法的响应。

由于分解机经常与稀疏数据一起使用，因此以CSV格式发出推理请求（如在其他算法示例中所做的那样）可能非常低效。为了浪费行和时间来生成所有这些零，可以将行填充到正确的维数，而可以更有效地使用JSON。

尽管如此，我们将编写自己的小函数以Amazon SageMaker Factorization Machines期望的JSON格式序列化我们的推理请求。

In [ ]:
import json
from sagemaker.predictor import json_deserializer

def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

### 邻近算法

让我们实现一个成对比较矩阵，该矩阵为稀疏数据集建立索引，以便对于每个给定向量，我们都能获得前10个最相似的稀疏向量。一旦获得相似的vecor，就进行二元分类 **(Inference Call)** 将对他们进行运算以找出最终的电影推荐。

In [ ]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=10)

In [ ]:
%%time

# 由于本次研讨会的时间有限，我们只选择整个数据集的1/5，以便获得较小的索引来快速进行计算。
# 请使用以下5个knn.fit中的任何一个，并记住，每个运行5至10分钟。
# 您的电影推荐会根据您适合的电影而有所不同。

knn.fit(X_train[0:20000].toarray())
# knn.fit(X_train[20000:40000].toarray())
# knn.fit(X_train[40000:60000].toarray())
# knn.fit(X_train[60000:80000].toarray())
# knn.fit(X_train[80000:100004].toarray())

### 应用接口


In [ ]:
from IPython.core import *
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from collections import OrderedDict
from IPython.display import Javascript
from IPython.display import display

In [ ]:
sortedMovies = list(newDf['title'].sort_values())

In [ ]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+2)'))

* 请注意，电影的下拉菜单会延迟3到5秒
* 单击 ***推荐*** 按钮
* 向下滚动以查看电影推荐

In [ ]:
# 该代码将创建“n”个下拉菜单供用户输入观看的电影
n = 0
while n < 1:    
    n = int(input("Enter a non-zero number of movies to select: "))
    
dropDownList = []
for i in range(n):
    dropDownList.append(widgets.Dropdown(options=sortedMovies, description="Movie: "+str(i+1), disabled=False))

    
Button = widgets.Button(description="Recommend")
Button.on_click(run_all)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+1)'))
    

VBox(dropDownList)

In [ ]:
Button

In [ ]:
# %%time
# 这整个单元执行“推荐”按钮的功能，仍然需要添加

def recommend():
    # userIDs是n个672s的列表，因为训练集中的ID中已经有671个用户672表示新用户。
    userIDs = [672]*n
    # 指定n个5秒的列表表示用户喜欢所有电影（假设）并给予5星
    userRatings = [5]*n
    movieIDs = []
    # 下面的for循环将用户观看的电影的ID迭代地附加到movieIDs列表中。.
    for mn in movieNames:
        movieIDs.append(int(newDf.movieid[newDf['title'] == mn]))
    
    # 现在，以与训练/测试相同的格式创建一个数据框，并带有新输入的影片.
    tempDf = pd.DataFrame(
        OrderedDict({
            'userid': userIDs,
            'movieid': movieIDs,
            'rating': userRatings
        })
    )
    
    # 确保列顺序保持不变 (userId, movieId, rating)
    tempDf = tempDf[['userid', 'movieid', 'rating']]
    
    # 将上面的数据帧转换为稀疏矩阵格式，然后返回X和y。
    # 记住，由于我们将默认等级初始化为5，所以所有y元素均为 1（true）
    X, y = loadDataset(tempDf, len(tempDf), nbFeatures)

    
    # 请记住，X是用户观看的所有电影的稀疏向量。
    # 现在，对于每部电影，下面的for循环将列出所有5个最接近的电影索引。
    # NearestNeighbors是根据原始训练数据构建的
    recommendationIndexes = []
    for x in X.toarray():
        recommendationIndexes.append(knn.kneighbors([x], return_distance=False))

    # RecommendationIndexes最初是n * 5的2D数组，其中n是用户选择的电影数量。由于我们只关心索引，因此我们仅将2D列表展平以得到1D列表。
    recommendationIndexes = np.array(recommendationIndexes).flatten()



    #print(nbUsers)
    # 现在，基于recommendedIndexed，我们需要提取建议的ID。
    # 这可能看起来很棘手，但是非常简单。
    # 请转到loadDataset（）函数以查看电影ID的编码方式。
    recommendationIDs = []
    for i in recommendationIndexes:
        row = X_train[i].toarray()
        #print(row[0])
        # 从loadDataset函数中，我们知道在稀疏行中找到第二个1的出现表示电影。
        # 下面的行查找该行中所有值为1的地方。
        # 这将告诉有关稀疏矩阵用户的详细信息，他给哪部电影评分以及该电影的流派是什么。
        ii = np.where(row[0] == 1)[0]
        #print(ii)
        #print(ii[1]+1-nbUsers)
        # 还记得我们在loadDataset函数中将movieIds编码为（nbUsers + movieId-1）吗？
        # 下面的代码与查找movieId的公式相反。
        # ii [1]保留movieId编码的索引，将其加1，然后减去用户数将返回该数字，该数字将是电影的确切ID。
        recommendationIDs.append(ii[1]+1-nbUsers)
        #print(ii)

    # 我们的邻居很可能会返回与用户已经输入的完全相同的电影。因此，为了避免这种情况，我们从推荐电影中减去用户观看的电影。
    # 投射到场景中可确保所有返回的推荐都是唯一的，并且不推荐两次重复播放电影。
    finalMovies = set(recommendationIDs)
    #- set(list(tempDf['movieId']))

    # 与之前说明的相同，但是这次，我们需要过滤掉最近邻居返回的电影，而不是用户选择的电影。
    # 因此，我们现在需要创建一个输入格式矩阵，该矩阵将传递到我们的二进制分类器中。
    
    uIDs = [672]*len(finalMovies)
    rts = [5]*len(finalMovies)
    mIDs = list(finalMovies)


    finalDf = pd.DataFrame(
        {
            'userid': uIDs,
            'movieid': mIDs,
            'rating': rts
        }
    )


    # 确保列顺序保持不变.
    finalDf = finalDf[['userid', 'movieid', 'rating']]

    # 再次加载.
    finalX, finaly = loadDataset(finalDf, len(finalDf), nbFeatures)
    #print("OK Till here")
    # 我们有一个最近邻居的电影列表，现在我们需要分类新用户是否会观看那些最近的电影。
    # 为此，我们将得到1或0（二进制分类器，还记得吗？）
    predictions = []
    for array in np.array_split(finalX[0:6].toarray(), 1):
        result = fm_predictor.predict(array)
        #print(result)
        # 分类器返回标签以及得分。
        # 现在我们只需要标签，所以我们只收集标签
        predictions += [r['predicted_label'] for r in result['predictions']]

    predictions = np.array(predictions)

    #print("OK Till here as well")
    
    
    # 下面的4行添加所选（标签1）电影的movieIds，并追加到filtereMovies列表中。
    filteredMovies = []
    for p in range(len(predictions)):
        if predictions[p] > 0:
            filteredMovies.append(finalDf['movieid'].iloc[p])

    if (len(filteredMovies) < 1):
        print("No movies to recommend with that watch history")
        return 0



    else:        
        # 显示预期电影


        links = pd.read_csv(dirName+'/'+'links.csv',names=['movieid','imdbid','tmdbid'],header=0)
        print(links.head(5))
        # 由于我们现在有了已过滤电影的ID列表，因此我们需要创建一个包含标题和imdb ID的数据框.
        recommendedMovieLinks = []
        recommendedMovies = []
        print("filter movies : ",filteredMovies)
        for i in filteredMovies:
            recommendedMovies.append(list(newDf.title[newDf['movieid'] == i]))
            recommendedMovieLinks.append(list(links.imdbid[links['movieid'] == i]))
            print("i : ",i)
        print("movies link : ",recommendedMovieLinks)
        for i in range(len(recommendedMovieLinks)):
            # 将imdb链接转换为https格式。
            # 我们需要做zfill事情，因为imdb链接格式具有7位数字的链接标准，因此，如果有任何ID都不是7位数字，则会添加前导零。例如，1234变为0001234。
            print("below i : ",i)
            recommendedMovieLinks[i][0] = 'https://www.imdb.com/title/tt'+str(recommendedMovieLinks[i][0]).zfill(7)+'/'

        # 根据RecommendationMovies和RecommendationMovieLinks创建数据框
        recommendations_links = pd.DataFrame(
            {
                'Title': recommendedMovies,
                'Link': recommendedMovieLinks
            }
        )
        # 确保我们按“标题”和“链接”的顺序获取列
        recommendations_links = recommendations_links[['Title', 'Link']]
        # 清除列表方括号，这样我们就只能得到没有方括号的字符串。
        # 当前数据框的标题类似['Movie Title']，但我们不希望使用方括号。
        # 我们只需要电影标题。链接也是如此。
        recommendedMovies = recommendations_links['Title'].values
        recommendedLinks = recommendations_links['Link'].values
        for i in range(len(recommendedMovies)):
            recommendedMovies[i] = recommendedMovies[i][0]
            recommendedLinks[i] = recommendedLinks[i][0]
        return pd.DataFrame(
            OrderedDict({
                'Title': recommendedMovies,
                'Link': recommendedLinks
            })
        )

# 下面的函数可修改val，并通过添加val将其创建为html链接 <a href> 标签.
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)
        
movieNames = []
for dd in dropDownList:
    movieNames.append(dd.value)
# 调用上面定义的Recommendation（）函数，该函数返回带有Title和Link列的dataFrame.
recLinks = recommend()

# 异常处理，如果不建议使用电影，则为可能（分类器返回所有0标签的可能性）
try:
    recLinks.set_index('Title', inplace=True)
except:
    print("No movies for the given watch-history could be recommended")

# 以可点击的格式显示链接。
recLinks.style.format(make_clickable)

## Clean up

In [ ]:
# 这很重要，因为即使完成工作后，我们也不希望任何杂散的EC2实例运行。
import sagemaker

sagemaker.Session().delete_endpoint(fm_predictor.endpoint)